In [3]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [8]:
from parser import get_lyrics_for_song 
from database import Song

In [9]:
import pandas as pd

In [11]:
df = pd.read_csv('data/yandex_tracks_top100.csv')
df.rename(columns={'artist(s)': 'artist', 'name': 'track'}, inplace=True)
df.head()

,track,track_len,link,genre,artist,chart,Explicit_content,monthly_listens_total,artists_likes_total
0,Царица,3:35,https://music.yandex.ru//album/26605629/track/...,русская поп-музыка,['ANNA ASTI'],1,0,4847753,2120528
1,Плохая Девочка,2:15,https://music.yandex.ru//album/26697445/track/...,русский рэп,"['Винтаж', 'ТРАВМА', 'SKIDRI', 'DVRKLXGHT']",2,0,16677658,417013
2,Я плачу,1:54,https://music.yandex.ru//album/26607707/track/...,русская поп-музыка,['A.V.G'],3,0,4406023,287295
3,ASPHALT 8,2:15,https://music.yandex.ru//album/25361122/track/...,русский рэп,['MACAN'],4,0,5149399,2359899
4,Где прошла ты,2:48,https://music.yandex.ru//album/25163882/track/...,русский рэп,"['Кравц', 'Гио Пика']",5,0,9058111,1509523


In [25]:
x = 3

In [29]:
eval('x + 15.0001')

18.0001

In [5]:
df['artist(s)'].apply(lambda x: len(eval(x))).value_counts()

artist(s)
1    71
2    20
3     4
4     3
0     1
5     1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [36]:
import requests
import json
from bs4 import BeautifulSoup
from unidecode import unidecode
from tqdm import tqdm

In [34]:
unidecode('ASPHALT 8') 

'ASPHALT 8'

<IPython.core.display.Javascript object>

In [38]:
def get_lyrics_for_song(artist: str, track: str, BASE_URL_LYRICS: str = "https://www.musixmatch.com/lyrics/{}/{}") -> list[str] | AttributeError:
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}
    response = requests.get(BASE_URL_LYRICS.format('-'.join(artist.split()), '-'.join(track.split())), headers=headers)
    if response.status_code != 200:
        print(f'haha {response.status_code}')
        return ''

    soup = BeautifulSoup(response.content, "html.parser")
    print(soup.find_all('class="css-175oi2r r-13awgt0"'))
    soup = soup.find("div", {"css-175oi2r r-zd98yo": "true"})
    print(soup)
    lyrics = []
    for elem in soup:
        if line := elem.next_element.text.strip():
            lyrics.append(line)
    print(lyrics[0])
    try:
        return '\n'.join(lyrics)
    except AttributeError as err:
        return ''




In [48]:
API_KEY = '5e8771d5b88f246c0ca497d672868d0b'
URL_TRACK_ID = "https://api.musixmatch.com/ws/1.1/track.search"
URL_TRACK_LYRICS = "https://api.musixmatch.com/ws/1.1/track.lyrics.get"

In [87]:
get_lyrics_from_api('ANNA-ASTI', 'Царица')

161078084
200


['Все твои романы — тяжёлый вид спорта',
 'Каждый бывший выводил из зоны комфорта',
 'Каждый бывший — тренер личностного роста',
 'Ты стала хитрее, детка стала взрослой',
 '',
 'И на этой дискотеке ты не плачешь под утро',
 'Под любимые треки, маскируя тушь пудрой',
 'Своё сердце обточила в острые, как нож, грани',
 'И оно превратилось в драгоценный, но камень',
 '',
 'Теперь он пьяный по твоей вине',
 'Царица, царица',
 'Один лишь взгляд, и лютый холод по спине',
 'Он просто не может в тебе не раствориться',
 '',
 'Мальчик поплыл, мальчик попал',
 'А как он стесняется, а как он целуется',
 'Мальчик поплыл прям в центре зала',
 'Пусть он танцует, пока танцуется',
 '',
 '...',
 '',
 '******* This Lyrics is NOT for Commercial use *******']

In [86]:
def get_lyrics_from_api(artist: str, track: str):
    params_track = {"q_track": track, 'q_artist': artist, 'apikey': API_KEY}
    response = requests.get(URL_TRACK_ID, params=params_track).json()
    track_id = response['message']['body']['track_list'][0]['track']['commontrack_id']
    print(track_id)
    params_lyrics = {'commontrack_id': track_id, 'apikey': API_KEY}
    response_lyrics = requests.get(URL_TRACK_LYRICS, params=params_lyrics)
    print(response_lyrics.status_code)
    if response_lyrics.status_code != 200:
        print('wtf')
        return ''
    response_lyrics = response_lyrics.json()
    return response_lyrics['message']['body']['lyrics']['lyrics_body'].split('\n')

In [ ]:
for index, line in tqdm(df.iterrows()):
    df.loc[index, 'lyrics'] = get_lyrics_for_song(eval(line.artist)[0], line.track) if len(eval(line.artist)) == 1 else ''
    break
    

In [ ]:
for index, line in tqdm(df.iterrows()):
    df.loc[index, 'lyrics'] = get_lyrics_from_api(eval(line.artist)[0], line.track) if len(eval(line.artist)) == 1 else ''
    break